## Importing Libraries

In [ ]:
import pandas as pd                                                                #used for data manipulation and analysis
import numpy as np                                              #high performance multidimensional array processing package
import matplotlib.pyplot as plt                                                                         #visualisation tool
import seaborn as sns                                                                                   #visualisation tool
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

### Importing the dataset

In [ ]:
df = pd.read_csv("../input/loan-prediction-problem-dataset/train_u6lujuX_CVtuZ9i.csv")
df.head()

### Understanding the dataset

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
print(f"The dataset has {df.shape[0]} rows and {df.shape[1]} columns")

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
sns.pairplot(df,hue='Loan_Status')

## Exploratory Data Analysis

In [ ]:
Categorical_features=[feature for feature in df.columns if df[feature].dtypes == 'object']
Categorical_features

In [ ]:
Numerical_features=[feature for feature in df.columns if df[feature].dtypes != 'object']
Numerical_features

In [ ]:
Continous_features=[]
Discrete_features=[]
for feature in Numerical_features:
    print(f'Number of unique values in column {feature} is {df[feature].nunique()}')
    if df[feature].nunique()>15:
        Continous_features.append(feature)
    else:
        Discrete_features.append(feature)

In [ ]:
#Taking columns having unique values less than 15 as discrete columns
Continous_features

In [ ]:
Discrete_features

In [ ]:
for feature in Categorical_features[1:]:
    plt.figure(figsize=(12,6))
    sns.countplot(data=df,x=feature,palette="rainbow",hue='Loan_Status')
    plt.title(f"{feature} Countplot")
    plt.tight_layout()
    plt.xlabel("")
    plt.show()

In [ ]:
for feature in Discrete_features:
    plt.figure(figsize=(12,6))
    sns.countplot(data=df,x=feature,palette="rainbow",hue='Loan_Status')
    plt.title(f"{feature} Countplot")
    plt.tight_layout()
    plt.xlabel("")
    plt.show()

In [ ]:
for feature in Continous_features:
    plt.figure(figsize=(12,6))
    sns.histplot(df[feature],palette="rainbow",kde=True)
    plt.title(f"{feature} Distribution")
    plt.tight_layout()
    plt.show()

In [ ]:
#missing values
for feature in df.columns:
    print(f'No. of missing values in column {feature} is {sum(df[feature].isnull())}')
    print(f'NUll value percentage : {round(np.mean(df[feature].isnull())*100,2)} % \n')

In [ ]:
#it can be seen that all the columns except Credit_history have less than 5 percent of null value
#can be easily be substituted by suitable central tendacy 
#in case of categorical and discrete mode will be suitable
#while for continous values we can make use of boxplot to make the decision
plt.figure(figsize=(15,15),dpi=150)
sns.heatmap(df.isnull(),yticklabels=False,cbar=False,cmap='magma')

In [ ]:
for feature in Categorical_features+Discrete_features:
    df[feature].fillna(df[feature].mode()[0],inplace=True)

In [ ]:
for feature in Continous_features:
    plt.figure(figsize=(12,6))
    sns.boxplot(data = df, y=feature)
    plt.title(f"{feature}")
    plt.tight_layout()
    plt.show()

In [ ]:
#since there are large numbers of outliers we are gonna fill the null values with median

In [ ]:
for feature in Continous_features:
    df[feature].fillna(np.nanmedian(df[feature]),inplace=True)

In [ ]:
plt.figure(figsize=(15,15),dpi=150)
sns.heatmap(df.isnull(),yticklabels=False,cbar=False,cmap='magma')

In [ ]:
#understanding the coleration

In [ ]:
df.corr()

In [ ]:
plt.figure(figsize=(15,15))
sns.heatmap(df.corr(),annot=True,cmap='rainbow')

In [ ]:
df.head()

In [ ]:
#Feature Engineering

In [ ]:
#creating a copy and working o new dataframe
data=df.copy()

In [ ]:
data.columns

In [ ]:
data["Total_Income"]=data["ApplicantIncome"]+data["CoapplicantIncome"]

In [ ]:
data.drop(['Loan_ID','ApplicantIncome', 'CoapplicantIncome','Dependents'],inplace=True,axis=1)

In [ ]:
Encoded_data=pd.DataFrame()

In [ ]:
for feature in data.columns:
    if(data[feature].nunique()>3):
        pass
    else:
        for i in range(data[feature].nunique()-1):
            Encoded_data[feature]=np.where(str(data[feature].unique()[i]) == data[feature],1,0)

In [ ]:
Encoded_data["LoanAmount"]=data["LoanAmount"]
Encoded_data["Loan_Amount_Term"]=data["Loan_Amount_Term"]
Encoded_data["Total_Income"]=data["Total_Income"]

In [ ]:
Encoded_data["Credit_History"]=data["Credit_History"]


In [ ]:
Encoded_data.head()

In [ ]:
Standard_data=Encoded_data.copy()
Standard_data['LoanAmount']=(Standard_data['LoanAmount']-Standard_data['LoanAmount'].min())/(Standard_data['LoanAmount'].max()-Standard_data['LoanAmount'].min())
Standard_data['Loan_Amount_Term']=(Standard_data['Loan_Amount_Term']-Standard_data['Loan_Amount_Term'].min())/(Standard_data['Loan_Amount_Term'].max()-Standard_data['Loan_Amount_Term'].min())
Standard_data['Total_Income']=(Standard_data['Total_Income']-Standard_data['Total_Income'].min())/(Standard_data['Total_Income'].max()-Standard_data['Total_Income'].min())
Standard_data.head()

In [ ]:
#we have scaled our data 
#since output is not given for test dataset 
#to check the performance we are gonna split our train dataset and work 
x = Standard_data.drop(columns="Loan_Status", axis=1)
y = Standard_data['Loan_Status']

In [ ]:
x_train, x_cv, y_train, y_cv = train_test_split(x, y, test_size=0.2)

In [ ]:
#using logistic regression model
loreg = LogisticRegression()

loreg.fit(x_train, y_train)
y_pred=loreg.predict(x_cv)

In [ ]:
print("Train Accuracy: ", loreg.score(x_cv, y_cv) * 100,' %')

In [ ]:
print(classification_report(y_cv,y_pred))

In [ ]:
print(confusion_matrix(y_cv,y_pred))

In [ ]:
score = cross_val_score(loreg, x, y, cv=5)
print("Cross validation is",np.mean(score)*100)